# RAG Evaluation with Ragas

In [17]:
import pandas as pd
from ragas import evaluate, EvaluationDataset

pd.set_option('display.max_colwidth', None)

In [ ]:
import json
with open('test_questions.json', 'r') as f:
    questions = json.load(f)

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "..."

In [20]:
# Evaluator LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")

In [21]:
# Embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-exp-03-07",
    task_type="retrieval_document"
)

In [22]:
dataset = []
count = 0
limit = 5

for question in questions:
    count += 1
    if count > limit:
        break
    dataset.append({
        'user_input': question['question'],
        'retrieved_contexts': ["this is a test context"],
        'response': "ศ.ดร.วิวัฒน์ วัฒนาวุฒิ",
        'reference': question['ground_truth'],
    })

In [23]:
eval_dataset = EvaluationDataset.from_list(dataset)

print(eval_dataset)
print(eval_dataset[0])

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=5)
user_input='อาจารย์ท่านใดบ้างที่จบปริญญาตรีวิศวกรรมคอมพิวเตอร์จากจุฬาฯ' retrieved_contexts=['this is a test context'] reference_contexts=None response='ศ.ดร.วิวัฒน์ วัฒนาวุฒิ' multi_responses=None reference='ศ.ดร.วิวัฒน์ วัฒนาวุฒิ, รศ.ดร.เกริก ภิรมย์โสภา, รศ.ดร.ดวงดาว วิชาดากุล, รศ.ดร.ธนารัตน์ ชลิดาพงศ์, รศ.ดร.นัทที นิภานันท์, รศ.ดร.วีระ เหมืองสิน, ผศ.ดร.ณรงค์เดช กีรติพรานนท์, ผศ.ดร.ณัฐวุฒิ หนูไพโรจน์, ผศ.ดร.เนื่องวงศ์ ทวยเจริญ, ผศ.ดร.พิตติพล คันธวัฒน์, ผศ.ดร.สุกรี สินธุภิญโญ, ผศ.ดร.อรรถวิทย์ สุดแสง, อ.ดร.กมลลักษณ์ สุขเสน, อ.ดร.เจษฏา ธัชแก้วกรพินธุ์, ดร.ณัท ภวสันต์, อ.ดร.วริษา ศรีไตรรัตนรักษ์, อ.ดร.สุขุม สัตตรัตนามัย' rubrics=None


In [24]:
result = evaluate(eval_dataset, llm=llm, embeddings=embeddings)

print(result)

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 8
}
].
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded y

{'answer_relevancy': 0.6675, 'context_precision': 0.0000, 'faithfulness': 0.0000, 'context_recall': 0.0000}


In [25]:
results_df = result.to_pandas()
display(results_df)

,user_input,retrieved_contexts,response,reference,answer_relevancy,context_precision,faithfulness,context_recall
0,อาจารย์ท่านใดบ้างที่จบปริญญาตรีวิศวกรรมคอมพิวเตอร์จากจุฬาฯ,[this is a test context],ศ.ดร.วิวัฒน์ วัฒนาวุฒิ,"ศ.ดร.วิวัฒน์ วัฒนาวุฒิ, รศ.ดร.เกริก ภิรมย์โสภา, รศ.ดร.ดวงดาว วิชาดากุล, รศ.ดร.ธนารัตน์ ชลิดาพงศ์, รศ.ดร.นัทที นิภานันท์, รศ.ดร.วีระ เหมืองสิน, ผศ.ดร.ณรงค์เดช กีรติพรานนท์, ผศ.ดร.ณัฐวุฒิ หนูไพโรจน์, ผศ.ดร.เนื่องวงศ์ ทวยเจริญ, ผศ.ดร.พิตติพล คันธวัฒน์, ผศ.ดร.สุกรี สินธุภิญโญ, ผศ.ดร.อรรถวิทย์ สุดแสง, อ.ดร.กมลลักษณ์ สุขเสน, อ.ดร.เจษฏา ธัชแก้วกรพินธุ์, ดร.ณัท ภวสันต์, อ.ดร.วริษา ศรีไตรรัตนรักษ์, อ.ดร.สุขุม สัตตรัตนามัย",0.66751,0.0,0.0,0.0
1,อาจารย์ท่านใดบ้างที่จบการศึกษาจากประเทศญี่ปุ่น,[this is a test context],ศ.ดร.วิวัฒน์ วัฒนาวุฒิ,"ศ.ดร.บุญเสริม กิจศิริกุล, รศ.ดร.กุลธิดา โรจน์วิบูลย์ชัย, รศ.ดร.พิษณุ คนองชัยยศ, ดร.ณัท ภวสันต์, ผศ.ดร.อาทิตย์ ทองทักษ์, ดร.สุธี เรืองวิเศษ",NaN,0.0,NaN,0.0
2,อาจารย์ท่านใดบ้างที่จบการศึกษาจาก Massachusetts Institute of Technology (MIT),[this is a test context],ศ.ดร.วิวัฒน์ วัฒนาวุฒิ,"ผศ.ดร.เอกพล ช่วงสุวนิช, รศ.ดร.อติวงศ์ สุชาโต, ดร.สุธี เรืองวิเศษ",NaN,0.0,0.0,0.0
3,อาจารย์ท่านใดบ้างที่จบการศึกษาจากประเทศอังกฤษ,[this is a test context],ศ.ดร.วิวัฒน์ วัฒนาวุฒิ,"ศ.ดร.ประภาส จงสถิตย์วัฒนา, รศ.ดร.ทวิตีย์ เสนีวงศ์ ณ อยุธยา, รศ.ดร.เศรษฐา ปานงาม, รศ.ดร.วีระ เหมืองสิน, รศ.ดร.วิษณุ โคตรจรัส, อ.ดร.มัชฌิกา อ่องแตง",NaN,0.0,NaN,0.0
4,อาจารย์ท่านใดที่มีความสนใจด้าน Artificial Intelligence หรือ Machine Learning,[this is a test context],ศ.ดร.วิวัฒน์ วัฒนาวุฒิ,"ศ.ดร.บุญเสริม กิจศิริกุล, ศ.ดร.ประภาส จงสถิตย์วัฒนา, รศ.ดร.โชติรัตน์ รัตนามหัทธนะ, รศ.ดร.พีรพล เวทีกูล, รศ.ดร. วิษณุ โคตรจรัส, ผศ.ดร.ณรงค์เดช กีรติพรานนท์, ผศ.ดร.พรรณราย ศิริเจริญ, ผศ.ดร.พิตติพล คันธวัฒน์, ผศ.ดร. สุกรี สินธุภิญโญ, ผศ.ดร. เอกพล ช่วงสุวนิช, อ.ดร.เจษฏา ธัชแก้วกรพินธุ์, ดร.ณัท ภวสันต์, อ.ดร.วริษา ศรีไตรรัตนรักษ์",NaN,0.0,0.0,0.0
